In [ ]:
import os
import textwrap
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
ds = load_dataset(
    "hwaseem04/Aya-testing",
    data_files={"xGQA_vqa": "data/xGQA_vqa-00000-of-00001.parquet"}
)

In [ ]:
ds['xGQA_vqa'][0]


In [ ]:
# Standard setup
from tqdm import tqdm
from PIL import Image
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

# Load Gemma-3 model and processor
model_id = "google/gemma-3-4b-it"
model = Gemma3ForConditionalGeneration.from_pretrained(model_id, device_map="auto").eval()
processor = AutoProcessor.from_pretrained(model_id)

# Languages to iterate over
languages = ["en", "bn", "de", "ko", "ru", "zh"]

# Directory to save temp images (needed for this model)
os.makedirs("temp_images_vqa", exist_ok=True)

dataset = ds['xGQA_vqa']

for sample in tqdm(dataset, desc="Iterating samples"):
    try:
        image = sample["image"]
        sample_id = sample["sample_id"]

        # Save the image locally if not already present
        image_path = f"temp_images_vqa/{sample_id}.jpg"
        if not os.path.exists(image_path):
            image.save(image_path)

        print(f"\n========== Sample ID: {sample_id} ==========")

        for lang in languages:
            question_col = f"question_{lang}"
            answer_col = f"answer_{lang}"

            # Safety check if expected keys exist
            if question_col not in sample or answer_col not in sample:
                print(f"[{lang}] Missing data.")
                continue

            question = sample[question_col]
            gt_answer = sample[answer_col]

            # Create chat-format input as expected by Gemma
            messages = [
                {
                    "role": "system",
                    "content": [{"type": "text", "text": "You are a helpful assistant."}]
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image_path},
                        {"type": "text", "text": question}
                    ]
                }
            ]

            # Format input for the model
            inputs = processor.apply_chat_template(
                messages,
                add_generation_prompt=True,
                tokenize=True,
                return_dict=True,
                return_tensors="pt"
            ).to(model.device, dtype=torch.bfloat16)

            input_len = inputs["input_ids"].shape[-1]

            # Generate prediction using Gemma
            with torch.inference_mode():
                generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
                generation = generation[0][input_len:]

            # Decode output
            pred_answer = processor.decode(generation, skip_special_tokens=True)

            # Display result
            print(f"\n[{lang.upper()}]")
            print(f"Question: {question}")
            print(f"GT: {gt_answer}")
            print(f"Pred: {textwrap.fill(pred_answer, width=80)}")

        print("=" * 100)

    except Exception as e:
        print(f"Error processing sample {sample['sample_id']}: {e}")

    break